# NOP NOP Trajectories Bug Testing

Includes functionality to:
1. Read encounter information straight from the bin files
2. Simulate encounter files
3. Visualize those simulations (if not small_simulation_output())

**Make sure you are in a Julia 1.1 Env!
And that you have aircraftshapes.sty in repo**

In [17]:
include("src/Xr.jl");

In [41]:
# Generate new encounters from within the Jupyter notebook
@time cd("../UAMEncounterGen") do
    include("generate_nop_nop_encs.jl")
    cp(filename1, joinpath("../XrSim/data_files", basename(filename1)), force=true)
    cp(filename2, joinpath("../XrSim/data_files", basename(filename2)), force=true)
end

 28.200312 seconds (5.49 M allocations: 7.264 GiB, 4.71% gc time)


┌ Warning: encounter_files/nop_nop_UstarUstar_flip_test_1z.bin already exists... overwriting!
└ @ Main C:\Users\mossr\Code\sisl\ACAS Xr\UAMEncounterGen\generate_nop_nop_encs.jl:15


"../XrSim/data_files/nop_nop_UstarUstar_flip_test_2z.bin"

### 1. Simulating the encounter files

Make sure this notebook is within the UAMEncounterGen repo. Make sure that your encounter files are in the "data_files" folder within UAMEncounterGenerator. 

When running a simulation, to see alerts or nmacs, must do sim.sim_out = small_simulation_output()

When running a simulation, to be able to visualize it, do not do a small simulation

sim 1 : trajectory 1 equipped as ac1

sim 2 : trajectory 1 equipped as ac2

sim 3 : trajectory 2 equipped as ac2

sim 4 : trajectory 2 equipped as ac1

In [42]:
run_vertical_logic = false

encounter_file1 = joinpath("data_files", basename(filename1))
encounter_file2 = joinpath("data_files", basename(filename2))

sim1 = simulation()

# comment this back in when wanting to see the alert counts
sim1.sim_out = small_simulation_output()

sim1.enc_file = encounter_file1
if run_vertical_logic
    sim1.acs[1] = uam_vert(q_file = "data_files/test.bin")
else
    sim1.acs[1] = uam_speed(q_file = "data_files/test_speed_120.bin")
    sim1.acs[1].perform_scaling = true
end

# turn responsive to false when wanting to check if trajectories are being properly generated
# sim1.acs[1].responsive = false

xr_sim!(sim1);

In [43]:
sim2 = simulation()
sim2.sim_out = small_simulation_output()
sim2.enc_file = encounter_file2
if run_vertical_logic
    sim2.acs[2] = uam_vert(q_file = "data_files/test.bin")
else
    sim2.acs[2] = uam_speed(q_file = "data_files/test_speed_120.bin")
    sim2.acs[2].perform_scaling = true
end
xr_sim!(sim2);

In [44]:
sim3 = simulation()
sim3.sim_out = small_simulation_output()
sim3.enc_file = encounter_file1
if run_vertical_logic
    sim3.acs[2] = uam_vert(q_file = "data_files/test.bin")
else
    sim3.acs[2] = uam_speed(q_file = "data_files/test_speed_120.bin")
    sim3.acs[2].perform_scaling = true
end
xr_sim!(sim3);

In [45]:
sim4 = simulation()
sim4.sim_out = small_simulation_output()
sim4.enc_file = encounter_file2
if run_vertical_logic
    sim4.acs[1] = uam_vert(q_file = "data_files/test.bin")
else
    sim4.acs[1] = uam_speed(q_file = "data_files/test_speed_120.bin")
    sim4.acs[1].perform_scaling = true
end
xr_sim!(sim4);

In [46]:
println(sim1.sim_out.alerts)
println(sim2.sim_out.alerts)
println()
println(sim3.sim_out.alerts)
println(sim4.sim_out.alerts)

295
295

348
348


### 2. Reading straight from the encounter files

In [ ]:
open("data_files/nop_nop_UstarUstar_flip_test_1z.bin", "r") do f
    dt = read(f, Float64)
    num_steps = read(f, Int64)
    # For each encounter
    for i = 1:2
        # Make an encounter out of info
        curr_phys_state = read_phys_state(f)
        ẍ = read_vector(f, num_steps)
        ÿ = read_vector(f, num_steps)
        z̈ = read_vector(f, num_steps)

        #@show curr_phys_state
        #@show ẍ
        #@show ÿ
        #@show z̈
        #@show length(ẍ)
    end
end


### 3. Visualizing the simulated encounter sets

Make sure the simulation you want to visualize does not have sim.sim_out = small_simulation_output()

ac1 = UAM icon

ac2 = quad icon

In [5]:
include("visualization/UAMEncounterViewer.jl");

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7105744843655464791\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-5127325886245359431\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7105744843655464791\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [ ]:
encounter_viewer(sim1.sim_out, int_type=:quad, alert_type=:speed)